# Data Preprocessing - Student Engagement Analytics

**Objective**: Load dataset from Google Drive, clean, and prepare features for Model 1

**Steps**:
1. Mount Google Drive
2. Load dataset (Merge.csv)
3. Exploratory Data Analysis (EDA)
4. Data Cleaning
5. Feature Engineering
6. Save Preprocessed Data

## 1. Setup and Mount Google Drive

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn scikit-learn -q

print("✅ Packages installed successfully")

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported successfully")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully")
print("\nPlease upload 'Merge.csv' to your Google Drive")
print("Suggested path: /content/drive/MyDrive/FYP_Data/Merge.csv")

## 2. Load Dataset

In [ ]:
# Load dataset from Google Drive
# MODIFY THIS PATH TO MATCH YOUR GOOGLE DRIVE LOCATION
DATA_PATH = '/content/drive/MyDrive/FYP_Data/Merge.csv'

try:
    df = pd.read_csv(DATA_PATH)
    print("✅ Dataset loaded successfully!")
    print(f"\nDataset Shape: {df.shape}")
    print(f"Total Records: {len(df)}")
    print(f"Total Features: {len(df.columns)}")
except FileNotFoundError:
    print("❌ File not found! Please check the path and upload Merge.csv to Google Drive")
    print(f"Looking for file at: {DATA_PATH}")

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
print("="*80)
df.info()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("Statistical Summary:")
print("="*80)
df.describe()

In [ ]:
# Check missing values
print("Missing Values:")
print("="*80)
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing_Count': missing,
    'Percentage': missing_percent
})
print(missing_df[missing_df['Missing_Count'] > 0])

if missing_df['Missing_Count'].sum() == 0:
    print("✅ No missing values found!")

In [ ]:
# Target variable distribution
print("Target Variable: Engagement Level")
print("="*80)
print("\nDistribution:")
print(df['Engagement Level'].value_counts())
print("\nPercentage:")
print(df['Engagement Level'].value_counts(normalize=True) * 100)

# Visualize
plt.figure(figsize=(10, 6))
df['Engagement Level'].value_counts().plot(kind='bar', color=['#6BCB77', '#FFD93D', '#FF6B6B'])
plt.title('Engagement Level Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Engagement Level', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n⚠️ Note: Dataset is imbalanced (Active: 63%, Passive: 31.5%, Moderate: 5.5%)")
print("We'll handle this with class weights in the model")

In [ ]:
# Analyze key features
print("Key Feature Analysis:")
print("="*80)

# Student statistics
print(f"\nTotal Students: {df['Student Name'].nunique()}")
print(f"Total Quizzes: {df['Quiz#'].nunique()}")
print(f"Total Questions: {df['Question ID'].nunique()}")

# Network quality distribution
print("\nNetwork Quality Distribution:")
print(df['Network Quality'].value_counts())

# Correctness statistics
print("\nAnswer Correctness:")
print(df['Is Correct'].value_counts())

In [ ]:
# Visualize key numerical features
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribution of Key Features', fontsize=16, fontweight='bold')

# Response Time
axes[0, 0].hist(df['Response Time (sec)'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Response Time (sec)')
axes[0, 0].set_xlabel('Time (seconds)')
axes[0, 0].set_ylabel('Frequency')

# RTT
axes[0, 1].hist(df['RTT (ms)'], bins=50, color='lightgreen', edgecolor='black')
axes[0, 1].set_title('RTT (ms)')
axes[0, 1].set_xlabel('RTT (ms)')
axes[0, 1].set_ylabel('Frequency')

# Jitter
axes[0, 2].hist(df['Jitter (ms)'], bins=50, color='lightcoral', edgecolor='black')
axes[0, 2].set_title('Jitter (ms)')
axes[0, 2].set_xlabel('Jitter (ms)')
axes[0, 2].set_ylabel('Frequency')

# Stability
axes[1, 0].hist(df['Stability (%)'], bins=50, color='gold', edgecolor='black')
axes[1, 0].set_title('Stability (%)')
axes[1, 0].set_xlabel('Stability (%)')
axes[1, 0].set_ylabel('Frequency')

# Engagement by Network Quality
pd.crosstab(df['Network Quality'], df['Engagement Level']).plot(kind='bar', ax=axes[1, 1], color=['#6BCB77', '#FFD93D', '#FF6B6B'])
axes[1, 1].set_title('Engagement by Network Quality')
axes[1, 1].set_xlabel('Network Quality')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

# Hide last subplot
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 4. Data Cleaning

In [ ]:
# Create a copy for preprocessing
df_clean = df.copy()

print("Starting Data Cleaning...")
print("="*80)

# 1. Handle any missing values (if present)
print("\n1. Checking for missing values...")
if df_clean.isnull().sum().sum() > 0:
    print("   Found missing values. Filling...")
    # Fill numeric columns with median
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    df_clean[numeric_cols] = df_clean[numeric_cols].fillna(df_clean[numeric_cols].median())
    
    # Fill categorical with mode
    categorical_cols = df_clean.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)
    print("   ✅ Missing values handled")
else:
    print("   ✅ No missing values found")

# 2. Remove duplicates
print("\n2. Checking for duplicates...")
duplicates = df_clean.duplicated().sum()
if duplicates > 0:
    print(f"   Found {duplicates} duplicate rows. Removing...")
    df_clean = df_clean.drop_duplicates()
    print("   ✅ Duplicates removed")
else:
    print("   ✅ No duplicates found")

# 3. Handle outliers (optional - using IQR method for key features)
print("\n3. Analyzing outliers in Response Time...")
Q1 = df_clean['Response Time (sec)'].quantile(0.25)
Q3 = df_clean['Response Time (sec)'].quantile(0.75)
IQR = Q3 - Q1
outliers = ((df_clean['Response Time (sec)'] < (Q1 - 1.5 * IQR)) | 
            (df_clean['Response Time (sec)'] > (Q3 + 1.5 * IQR))).sum()
print(f"   Found {outliers} outliers in Response Time")
print(f"   Keeping outliers as they represent real behavioral patterns")

print("\n" + "="*80)
print(f"✅ Data Cleaning Complete")
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_clean.shape}")

## 5. Feature Engineering

In [ ]:
print("Feature Engineering...")
print("="*80)

# 1. Convert 'Is Correct' to binary
print("\n1. Converting 'Is Correct' to binary...")
df_clean['Is_Correct_Binary'] = df_clean['Is Correct'].apply(
    lambda x: 1 if str(x).lower() == 'yes' else 0
)
print("   ✅ Done")

# 2. Encode Engagement Level (target)
print("\n2. Encoding Engagement Level...")
engagement_mapping = {'Passive': 0, 'Moderate': 1, 'Active': 2}
df_clean['Engagement_Encoded'] = df_clean['Engagement Level'].map(engagement_mapping)
print("   Mapping: Passive=0, Moderate=1, Active=2")
print("   ✅ Done")

# 3. Encode Network Quality
print("\n3. Encoding Network Quality...")
network_mapping = {'Poor': 0, 'Fair': 1, 'Good': 2, 'Excellent': 3}
df_clean['Network_Quality_Encoded'] = df_clean['Network Quality'].map(network_mapping)
# Handle any unmapped values
if df_clean['Network_Quality_Encoded'].isnull().any():
    print("   Warning: Some network quality values not mapped. Filling with 1 (Fair)")
    df_clean['Network_Quality_Encoded'].fillna(1, inplace=True)
print("   ✅ Done")

# 4. Calculate student-level aggregated features
print("\n4. Creating student-level aggregated features...")

# Average accuracy per student
student_accuracy = df_clean.groupby('Student Name')['Is_Correct_Binary'].mean().reset_index()
student_accuracy.columns = ['Student Name', 'Student_Avg_Accuracy']
df_clean = df_clean.merge(student_accuracy, on='Student Name', how='left')

# Average response time per student
student_response_time = df_clean.groupby('Student Name')['Response Time (sec)'].mean().reset_index()
student_response_time.columns = ['Student Name', 'Student_Avg_Response_Time']
df_clean = df_clean.merge(student_response_time, on='Student Name', how='left')

# Average RTT per student
student_rtt = df_clean.groupby('Student Name')['RTT (ms)'].mean().reset_index()
student_rtt.columns = ['Student Name', 'Student_Avg_RTT']
df_clean = df_clean.merge(student_rtt, on='Student Name', how='left')

print("   ✅ Done")

# 5. Create time-based features
print("\n5. Creating time-based features...")
df_clean['Answered_At_Datetime'] = pd.to_datetime(df_clean['Answered At'])
df_clean['Hour_of_Day'] = df_clean['Answered_At_Datetime'].dt.hour
df_clean['Day_of_Week'] = df_clean['Answered_At_Datetime'].dt.dayofweek
print("   ✅ Done")

print("\n" + "="*80)
print("✅ Feature Engineering Complete")
print(f"Total features now: {len(df_clean.columns)}")

In [ ]:
# Display new features
print("New Features Created:")
print("="*80)
new_features = ['Is_Correct_Binary', 'Engagement_Encoded', 'Network_Quality_Encoded',
                'Student_Avg_Accuracy', 'Student_Avg_Response_Time', 'Student_Avg_RTT',
                'Hour_of_Day', 'Day_of_Week']
df_clean[new_features].head(10)

## 6. Feature Selection for Models

In [ ]:
print("Selecting Features for Models...")
print("="*80)

# Features for Model 1 (Clustering + Prediction)
feature_columns = [
    # Response metrics
    'Response Time (sec)',
    'Response Time (ms)',
    
    # Network metrics
    'RTT (ms)',
    'Jitter (ms)',
    'Stability (%)',
    'Network_Quality_Encoded',
    
    # Performance metrics
    'Is_Correct_Binary',
    'Student_Avg_Accuracy',
    'Student_Avg_Response_Time',
    'Student_Avg_RTT',
    
    # Additional features
    'Hour_of_Day',
    'Day_of_Week'
]

# Target variable
target_column = 'Engagement_Encoded'

print(f"\nSelected {len(feature_columns)} features for modeling:")
for i, col in enumerate(feature_columns, 1):
    print(f"  {i}. {col}")

print(f"\nTarget variable: {target_column} (Engagement Level)")
print("\n✅ Feature selection complete")

In [ ]:
# Correlation analysis
print("Correlation Analysis with Target Variable:")
print("="*80)

# Calculate correlations
correlations = df_clean[feature_columns + [target_column]].corr()[target_column].sort_values(ascending=False)
print(correlations[1:])  # Exclude self-correlation

# Visualize correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df_clean[feature_columns + [target_column]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f', square=True)
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Save Preprocessed Data

In [ ]:
# Save preprocessed data to Google Drive
OUTPUT_PATH = '/content/drive/MyDrive/FYP_Data/'

print("Saving Preprocessed Data...")
print("="*80)

# 1. Save full preprocessed dataset
full_output_path = OUTPUT_PATH + 'Merge_Preprocessed.csv'
df_clean.to_csv(full_output_path, index=False)
print(f"✅ Full preprocessed data saved: {full_output_path}")

# 2. Save feature matrix and target
X = df_clean[feature_columns]
y = df_clean[target_column]

features_output_path = OUTPUT_PATH + 'Features_X.csv'
target_output_path = OUTPUT_PATH + 'Target_y.csv'

X.to_csv(features_output_path, index=False)
y.to_csv(target_output_path, index=False)

print(f"✅ Feature matrix saved: {features_output_path}")
print(f"✅ Target variable saved: {target_output_path}")

# 3. Save student-level aggregated data
student_data = df_clean.groupby('Student Name').agg({
    'Student_Avg_Accuracy': 'first',
    'Student_Avg_Response_Time': 'first',
    'Student_Avg_RTT': 'first',
    'RTT (ms)': 'mean',
    'Jitter (ms)': 'mean',
    'Stability (%)': 'mean',
    'Network_Quality_Encoded': 'mean',
    'Engagement_Encoded': lambda x: x.mode()[0] if len(x.mode()) > 0 else x.iloc[0]
}).reset_index()

student_output_path = OUTPUT_PATH + 'Student_Aggregated_Data.csv'
student_data.to_csv(student_output_path, index=False)
print(f"✅ Student-level data saved: {student_output_path}")

print("\n" + "="*80)
print("✅ All preprocessed data saved successfully!")
print("\nSummary:")
print(f"  - Full dataset: {len(df_clean)} records, {len(df_clean.columns)} features")
print(f"  - Feature matrix: {X.shape}")
print(f"  - Target vector: {y.shape}")
print(f"  - Student-level data: {len(student_data)} students")

## 8. Data Summary for Model Training

In [ ]:
print("="*80)
print("DATA PREPROCESSING SUMMARY")
print("="*80)

print("\n📊 Dataset Overview:")
print(f"  - Total Records: {len(df_clean)}")
print(f"  - Total Students: {df_clean['Student Name'].nunique()}")
print(f"  - Total Features: {len(feature_columns)}")
print(f"  - Target Classes: {df_clean['Engagement Level'].nunique()}")

print("\n🎯 Target Distribution:")
for level, count in df_clean['Engagement Level'].value_counts().items():
    percentage = (count / len(df_clean)) * 100
    print(f"  - {level}: {count} ({percentage:.1f}%)")

print("\n🔧 Features Created:")
print(f"  - Binary features: 1 (Is_Correct_Binary)")
print(f"  - Encoded features: 2 (Engagement, Network Quality)")
print(f"  - Aggregated features: 3 (Student averages)")
print(f"  - Time features: 2 (Hour, Day of Week)")

print("\n💾 Saved Files:")
print(f"  1. Merge_Preprocessed.csv - Full preprocessed dataset")
print(f"  2. Features_X.csv - Feature matrix for modeling")
print(f"  3. Target_y.csv - Target variable")
print(f"  4. Student_Aggregated_Data.csv - Student-level aggregated data")

print("\n✅ Ready for Model 1 Training!")
print("\nNext Steps:")
print("  1. Open 'Model1_Clustering_Prediction.ipynb'")
print("  2. Load preprocessed data from Google Drive")
print("  3. Train clustering model (unsupervised)")
print("  4. Train prediction model (supervised)")
print("  5. Evaluate and validate models")
print("="*80)